# 2.8: 量子哈密顿量基态计算

量子哈密顿量$\hat{H}$：描述量子系统"力学"(mechanism)的核心算符。

> 力学指使用数序描述物理系统的演化规律

(定态)薛定谔方程：

$\hat{H}|\varphi\rangle = E|\varphi\rangle$

其中 $E = \langle\varphi|\hat{H}|\varphi\rangle$ 为能量，上述方程可化为本征问题。

> $E$ 的推导:
> 
> $\hat{H}|\varphi\rangle = E|\varphi\rangle$ 两边同乘 $\langle\varphi|$，得到 $E = \langle\varphi|\hat{H}|\varphi\rangle$，根据量子态的归一化性质 $\langle\varphi|\varphi\rangle = 1$，可得 $E = \langle\varphi|\hat{H}|\varphi\rangle$。

> $E$ 也是 $\hat{H}$ 的量子期望

最关心最小（代数）本征值问题——**基态**(ground state)问题：

$|\varphi_{GS}\rangle = \text{argmin}_{|\varphi\rangle}\langle\varphi|\hat{H}|\varphi\rangle$

> 关心最小代数本征值的原因：
>
> 本征值对应能量，能量越小，这个态出现的概率越大。温度越低，态的概率的差距越大。当温度趋近于绝对零度时，基态的概率趋近于1，其他态的概率趋近于0。也就是基态是最稳定的态。

TODO: 看书里 3.1 - 3.4 节对于基态的概率意义的解释

## 例子：4 qubit 的一维海森堡模型

4个自旋（量子比特）构成的一维海森堡模型

$\hat{H} = \frac{1}{4}\sum_{n=0}^2 \hat{H}_{n,n+1}$

$= \frac{1}{4}\sum_{n=0}^2 (\hat{\sigma}_n^x\hat{\sigma}_{n+1}^x + \hat{\sigma}_n^y\hat{\sigma}_{n+1}^y + J\hat{\sigma}_n^z\hat{\sigma}_{n+1}^z)$

哈密顿量$\hat{H}$中的每一项 $\hat{H}_{n,n+1} = \hat{\sigma}_n^x\hat{\sigma}_{n+1}^x + \hat{\sigma}_n^y\hat{\sigma}_{n+1}^y + J\hat{\sigma}_n^z\hat{\sigma}_{n+1}^z$ 为二体算符，其系数为一个 $(4\times4)$ 的矩阵，作用于多体态的规则见2.5节。

> $J$ 叫各项异性参数
>
> 在这个例子里，如果 $J=0$ 那么只有 xy 相互作用
> 
> 如果 $J\gg 1$ 那么只有 z 相互作用，xy 相互作用可以忽略

哈密顿量本身的系数为$(2^4\times2^4)$的矩阵。

### 方法一：算出哈密顿量($2^4 \times 2^4$)的系数的矩阵

$\hat{H}_{01} \rightarrow \text{kron}(\hat{H}_{01},\hat{I},\hat{I})$

$\hat{H}_{12} \rightarrow \text{kron}(\hat{I},\hat{H}_{12},\hat{I})$

$\hat{H}_{23} \rightarrow \text{kron}(\hat{I},\hat{I},\hat{H}_{23})$

图示如下
![hamiltonian_example](./images/hamiltonian_example.png)

相加后使用`eigsh`等函数求解最小本征态
> 计算复杂度：$O(2^N \times 2^N)$

### 方法二：线性算子法

**关键步骤为定义线性映射**

$f(|\varphi\rangle) = \frac{1}{4}\sum_{n=0}^2 \hat{H}_{n,n+1}|\varphi\rangle$

> $\hat{H}$ 是 $4\times4$ 矩阵，因为是二体算符
>
> $|\varphi\rangle$ 是 $4\times4$ 矩阵，因为左指标是 2 qubit 态展开，右指标是剩下的 qubit 态展开

降低了指数级复杂度：对于上述例子，共需进行3次(4×4)与(4×4)间的矩阵乘法，计算复杂度为
$O(3\times4\times4\times4)$

对于长度为N的自旋链，计算复杂度为
$O((N-1)\times4\times4\times2^{N-2})\sim O(N2^N)$

### Code

In [1]:
# |default_exp tensor_gates.hamiltonians
# |export
import torch
from tensor_network.tensor_gates.functional import pauli_operator

一维海森堡模型

$$
\begin{align}
\hat{H} &= \frac{1}{4}\sum_{n=0}^{2} \hat{H}_{n,n+1} \\
&= \frac{1}{4}\sum_{n=0}^{2} (J_x\hat{\sigma}_n^x\hat{\sigma}_{n+1}^x + J_y\hat{\sigma}_n^y\hat{\sigma}_{n+1}^y + J_z\hat{\sigma}_n^z\hat{\sigma}_{n+1}^z)
\end{align}
$$

In [2]:
# |export
def heisenberg(
    *,
    jx: float | torch.Tensor,
    jy: float | torch.Tensor,
    jz: float | torch.Tensor,
    double_precision: bool = False,
    return_matrix: bool = False,
) -> torch.Tensor:
    """
    Calculate the Hamiltonian of the Heisenberg model.
    Args:
        jx: float | torch.Tensor, the coupling strength of the x-axis.
        jy: float | torch.Tensor, the coupling strength of the y-axis.
        jz: float | torch.Tensor, the coupling strength of the z-axis.
        double_precision: bool, whether to use double precision.
        return_matrix: bool, whether to return the Hamiltonian in matrix form. By default, the Hamiltonian is returned in tensor form.
    Returns:
        torch.Tensor, the Hamiltonian of the Heisenberg model.
    """
    pauli_x = pauli_operator(pauli="X", double_precision=double_precision)
    pauli_y = pauli_operator(pauli="Y", double_precision=double_precision)
    pauli_z = pauli_operator(pauli="Z", double_precision=double_precision)
    hamiltonian = (
        jx * torch.einsum("ab, ij -> aibj", pauli_x, pauli_x)
        + jy * torch.einsum("ab, ij -> aibj", pauli_y, pauli_y).real
        + jz * torch.einsum("ab, ij -> aibj", pauli_z, pauli_z)
    )
    hamiltonian /= 4
    if return_matrix:
        return hamiltonian.reshape(4, 4)
    else:
        return hamiltonian

In [3]:
from typing import List
from einops import einsum
import numpy as np
from tensor_network.tensor_gates.functional import gate_outer_product
from scipy.sparse.linalg import LinearOperator, eigsh

In [4]:
# |export tensor_gates.functional
def kron(*matrices: torch.Tensor) -> torch.Tensor:
    """
    Calculate the Kronecker product of a list of matrices.

    Args:
        *matrices: torch.Tensor, the matrices to be multiplied.
    Returns:
        torch.Tensor, the Kronecker product of the matrices.
    """
    assert len(matrices) >= 2, "At least two matrices are needed for Kronecker product"
    mat = torch.kron(matrices[0], matrices[1])
    for m in matrices[2:]:
        mat = torch.kron(mat, m)
    return mat

In [5]:
hamiltonian = heisenberg(jx=1.0, jy=1.0, jz=1.0, double_precision=False)
h_mat = hamiltonian.reshape(4, 4)
identity = torch.eye(2)

In [6]:
# calculate total hamiltonian by matrix ops
total_hamiltonian_mat = kron(h_mat, identity, identity)
total_hamiltonian_mat += kron(identity, h_mat, identity)
total_hamiltonian_mat += kron(identity, identity, h_mat)
# calculate total hamiltonian by tensor ops
total_hamiltonian = gate_outer_product(hamiltonian, identity, identity)
total_hamiltonian += gate_outer_product(identity, hamiltonian, identity)
total_hamiltonian += gate_outer_product(identity, identity, hamiltonian)

assert torch.allclose(total_hamiltonian.reshape(2**4, 2**4), total_hamiltonian_mat)

In [7]:
eigvalues = torch.linalg.eigvalsh(total_hamiltonian_mat)
print(f"获取完整 Hamiltonian 后调用 eigvalsh, 基态能量 = {eigvalues.min()}")

获取完整 Hamiltonian 后调用 eigvalsh, 基态能量 = -1.6160258054733276


In [8]:
h_np = hamiltonian.numpy()


def heisenberg_hamilt_on_state(state: np.ndarray):
    state = state.reshape(2, 2, 2, 2)
    after_state = einsum(state, h_np, "l0 l1 s2 s3, l0 l1 r0 r1 -> r0 r1 s2 s3")
    after_state += einsum(state, h_np, "s0 l0 l1 s3, l0 l1 r0 r1 -> s0 r0 r1 s3")
    after_state += einsum(state, h_np, "s0 s1 l0 l1, l0 l1 r0 r1 -> s0 s1 r0 r1")
    return after_state.flatten()


linear_fn = LinearOperator(shape=(2**4, 2**4), matvec=heisenberg_hamilt_on_state)
smallest_eigvalue, _ = eigsh(linear_fn, k=1, which="SA")
print(f"LinearOperator, 基态能量 = {smallest_eigvalue[0]}")

LinearOperator, 基态能量 = -1.6160253286361694
